# Final training notebook with optimized hyperparameters

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import os
from glob import glob
import pickle
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
import torchmetrics
from pytorch_lightning.loggers import TensorBoardLogger
from monai import losses
import albumentations as A
from copy import deepcopy
from pytorch_lightning.callbacks import Callback
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import WeightedRandomSampler
import random
import segmentation_models_pytorch as smp
import gc

In [2]:
torch.set_float32_matmul_precision('medium')

In [3]:
# paths and hyperparameters
name = '2DCNN_SeResNext'
folder = '/home/olli/Projects/Kaggle/Vesuvius'
folder_data = os.path.join(folder, 'Data_New', 'Preprocessed_1024_12_Masked')
folder_logs = os.path.join(folder, f'Logs_2D')
depth = 12
size = 1024
batch_size = 8
num_workers = 0
epochs = 14
learning_rate = 2e-4  # max lr to reach
pct_increase = 0.33  # percentage of epochs to increase the lr before decreasing
bce_loss_weight = 0.5  # how much to weight dice vs bce (0.5-0.5; 0.7-0.3; ...)
ink_loss_weight = 2

# Set up the cross-validation

In [4]:
fold_1_path = os.path.join(folder_data, '1', f'{size}')
fold_2_path = os.path.join(folder_data, '2', f'{size}')
fold_3_path = os.path.join(folder_data, '3', f'{size}')

In [5]:
scans_1 = glob(fold_1_path + '/*.pickle')
scans_2 = glob(fold_2_path + '/*.pickle')
scans_3 = glob(fold_3_path + '/*.pickle')

In [6]:
scans_1[0]

'/home/olli/Projects/Kaggle/Vesuvius/Data_New/Preprocessed_1024_12_Masked/1/1024/35.pickle'

In [7]:
folds = [scans_1, scans_2, scans_3]

In [8]:
# check number of scans
[len(i) for i in folds]

[107, 360, 84]

# Create 3 Train-/Valid-Datasets from the folds

In [9]:
'''Write a function that returns the different train/valid-datasets and -paths.
   Each fold once is the validation data, the rest is for training.'''


def create_train_valid():

    train_folds = ['tr_1', 'tr_2', 'tr_3']
    valid_folds = ['val_1', 'val_2', 'val_3']

    # assign each train/valid dataset the correct folds with i
    for i, train, valid in zip(range(3), train_folds, valid_folds):

        # copy the folds
        flds = deepcopy(folds)

        globals()[valid_folds[i]] = flds.pop(i)  # i'ths index is valid rest is for train
        globals()[train_folds[i]] = flds[0] + flds[1]  # only 0 and 1 left
        
    train_folds = [tr_1, tr_2, tr_3]
    valid_folds = [val_1, val_2, val_3]
    
    return train_folds, valid_folds

In [10]:
train_data, valid_data = create_train_valid()

In [11]:
# check the number of cropped scans in each dataset
[len(i) for i in train_data]

[444, 191, 467]

In [12]:
[len(i) for i in valid_data]

[107, 360, 84]

In [13]:
# make sure there is no valid file in the corresponding train dataset
for i in range(3):
    for v in valid_data[i]:
        for t in train_data[i]:
            if v == t:
                print('LEAK')

# Define Augmentations

In [14]:
train_aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.82),
    A.ShiftScaleRotate(p=0.47),
    A.OneOf([
            A.GaussNoise(var_limit=[10, 50]),
            A.GaussianBlur(),
            A.MotionBlur(),
            ], p=0.5),
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.32),
    A.CoarseDropout(max_holes=1, max_width=int(size * 0.3), max_height=int(size * 0.3), 
                        mask_fill_value=0, p=0.47),
    A.Normalize(mean=[0] * depth, std=[1] * depth)
])

In [15]:
# use valid aug to normalize the scan
valid_aug = A.Compose([
    A.Normalize(mean=[0] * depth, std=[1] * depth)
])

# Define the PyTorch Dataset

In [16]:
class Data(Dataset):
    def __init__(self, paths, transform=False):#, df=df):
        self.paths = paths
        self.transform = transform
        #self.df = df
        
        random.Random(42).shuffle(self.paths)
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        path = self.paths[index]
        
        with open(path, 'rb') as f:
            data = pickle.load(f)
            
        X, mask, y = data
        
        if self.transform:
            augmented_data = self.transform(image=X, masks=[mask, y])
            
            X = augmented_data['image']
            mask = augmented_data['masks'][0]
            y = augmented_data['masks'][1]
        
        mask = torch.tensor(mask / 255.).type(torch.float32)
        y = torch.tensor(y / 255.).type(torch.float32)
        
        X = torch.tensor(X).permute(2, 0, 1).type(torch.float32)
        
        return X, mask, y

# Create a Lightning Data Module from that

In [17]:
# create a lightning datamodule class

class DataModule(pl.LightningDataModule):
    
    def __init__(self, train_ds, valid_ds, batch_size=batch_size, num_workers=num_workers):
        
        super().__init__()

        self.batch_size=batch_size
        self.num_workers=num_workers
        
        self.train_ds = train_ds
        self.valid_ds = valid_ds
        
    def setup(self, stage):
        
        self.train_ds = self.train_ds
        self.valid_ds = self.valid_ds
        
        # set up the sampler with the shuffled training paths
        #files = [path.split('.')[-2].split('/')[-1] for path in self.train_ds.paths]
        #scans = [path.split('.')[-2].split('/')[-3] for path in self.train_ds.paths]
        #filenames = [scan + '_' + file for scan, file in zip(scans, files)]
        
        # sort the df to only have these with filenames in Index col
        #df_filtered = self.train_ds.df[self.train_ds.df['Index'].isin(filenames)]
        
        # now sort the df_filtered exactly how the filesnames are sorted
        #df_sorted = df_filtered.iloc[np.argsort([filenames.index(i) for i in df_filtered['Index']])]
        
        # now get the weights that are ordered correctly
        #self.weights = df_sorted['Weight'].values
        
    def train_dataloader(self):
        
        train_dl = DataLoader(self.train_ds,
                              batch_size=self.batch_size,
                              num_workers=self.num_workers,
                              shuffle=True)
                              #pin_memory=True)
        
        return train_dl
        
    def val_dataloader(self):
        
        valid_dl = DataLoader(self.valid_ds,
                              batch_size=self.batch_size,
                              num_workers=self.num_workers,)
                              #pin_memory=True)
        
        return valid_dl

# Define the model

In [18]:
model = smp.Unet('se_resnext50_32x4d', encoder_weights=None, in_channels=depth, classes=1)

In [19]:
summary(model, input_data=torch.randn(2, depth, size, size))

Layer (type:depth-idx)                             Output Shape              Param #
Unet                                               [2, 1, 1024, 1024]        --
├─SENetEncoder: 1-1                                [2, 12, 1024, 1024]       --
│    └─Sequential: 2-1                             --                        --
│    │    └─Conv2d: 3-1                            [2, 64, 512, 512]         37,632
│    │    └─BatchNorm2d: 3-2                       [2, 64, 512, 512]         128
│    │    └─ReLU: 3-3                              [2, 64, 512, 512]         --
│    │    └─MaxPool2d: 3-4                         [2, 64, 256, 256]         --
│    └─Sequential: 2-2                             [2, 256, 256, 256]        --
│    │    └─SEResNeXtBottleneck: 3-5               [2, 256, 256, 256]        71,952
│    │    └─SEResNeXtBottleneck: 3-6               [2, 256, 256, 256]        79,632
│    │    └─SEResNeXtBottleneck: 3-7               [2, 256, 256, 256]        79,632
│    └─Sequential:

In [20]:
# check output shape and make sure that no sigmoid activation is at the last layer
with torch.no_grad():
    out = model(torch.randn(1, depth, size, size))
    print(out.min(), out.max())
    print(out.size())

tensor(-4.5020) tensor(6.3678)
torch.Size([1, 1, 1024, 1024])


In [21]:
# save the initial model weights to use them for each fold
random_weights_path = f'{folder}/Weights/{name}_random.pth'
if not os.path.exists(random_weights_path):
    torch.save(model.state_dict(), random_weights_path)

# Create a custom metric for the dice F 0.5 score

In [22]:
class CustomDice(torchmetrics.Metric):
    def __init__(self, beta=0.5, smooth=1e-5):
        super().__init__()
        
        self.beta = beta
        self.smooth = smooth
        self.add_state("ctp", default=torch.tensor(0, dtype=torch.float32), dist_reduce_fx="sum")
        self.add_state("cfp", default=torch.tensor(0, dtype=torch.float32), dist_reduce_fx="sum")
        self.add_state("y_true_count", default=torch.tensor(0, dtype=torch.float32), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor, apply_sigmoid=True):
        
        if apply_sigmoid:
            preds = torch.sigmoid(preds)
        preds = preds.view(-1)#.float()
        target = target.view(-1)#.float()

        self.y_true_count += target.sum()
        self.ctp += preds[target == 1].sum()
        self.cfp += preds[target == 0].sum()

    def compute(self):
        c_precision = self.ctp / (self.ctp + self.cfp + self.smooth)
        c_recall = self.ctp / (self.y_true_count + self.smooth)
        beta_squared = self.beta * self.beta
        dice_score = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + self.smooth)

        return dice_score

# Create the Lightning Module

In [23]:
class NN(pl.LightningModule):
    def __init__(self, 
                 model, 
                 learning_rate,
                 overweight_ink=ink_loss_weight,
                 bce_loss_weight=bce_loss_weight,
                 calculate_threshold=False):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.dice = CustomDice()
        self.calc_th = calculate_threshold
        self.overweight_ink = ink_loss_weight
        self.bce_loss_weight = bce_loss_weight
        self.epochs = epochs
        self.pct_increase = pct_increase
        self.BCE = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(self.overweight_ink))
        
        self.dice_loss = losses.DiceLoss(sigmoid=True)
        #self.masked_dice = losses.MaskedDiceLoss(sigmoid=True)
        self.bce = nn.functional.binary_cross_entropy_with_logits
        
        self.predictions = []  # for holding both to calculate the optimal thresholds
        self.targets = []
        
    def forward(self, x):
        x = self.model(x)
        
        return x
    
    def training_step(self, batch, batch_idx):
        X, mask, y = batch
        
        pred = self.model(X)
        
        # add class dim
        mask = mask.unsqueeze(1)
        y = y.unsqueeze(1)
        
        dice_loss = self.dice_loss(pred, y)
        
        bce_loss = self.BCE(pred, y)
        
        #dice_loss = self.masked_dice(pred, target=y, mask=mask)
                
        #bce_loss = self.bce(pred, y, weight=mask)
        
        # now calculate the final loss on the defined ration (diceloss vs bceloss)
        loss = self.bce_loss_weight * bce_loss + (1 - self.bce_loss_weight) * dice_loss
        
        #loss = self.BCE(pred, y)
        
        with torch.no_grad():
            dice_score = self.dice(pred, y)
            
        self.log_dict({'train_loss': loss,
                       'train_dice': dice_score},
                       on_step=False,
                       on_epoch=True,
                       prog_bar=True,
                       logger=True)
        
        return {'loss': loss, 'train_dice': dice_score}
        
    def validation_step(self, batch, batch_idx):
        X, mask, y = batch
        
        with torch.no_grad():
            pred = self.model(X)
        
        # append the pred and y for thresholding
        self.predictions.append(pred.flatten()) # flatten them for calculation
        self.targets.append(y.flatten())
        
        # add class dim
        mask = mask.unsqueeze(1)
        y = y.unsqueeze(1)
        
        dice_loss = self.dice_loss(pred, y)
        
        bce_loss = self.BCE(pred, y)
        
        #dice_loss = self.masked_dice(pred, target=y, mask=mask)
                
        #bce_loss = self.bce(pred, y, weight=mask)
        
        # now calculate the final loss on the defined ration (diceloss vs bceloss)
        loss = self.bce_loss_weight * bce_loss + (1 - self.bce_loss_weight) * dice_loss
        
        #loss = self.BCE(pred, y)
        
        dice_score = self.dice(pred, y)
        
        self.log_dict({'valid_loss': loss,
                       'valid_dice': dice_score},
                        on_step=False,
                        on_epoch=True,
                        prog_bar=True,
                        logger=True)
        
        return {'valid_loss': loss, 'valid_dice': dice_score}
    
    def on_validation_epoch_end(self):
        #if self.calc_th:  # dont calculate the optimal threshold when tuning hp
            thresholds = np.arange(start=0.2, stop=0.8, step=0.05)  # thresholds to try
            scores = []  # append the scores here to log them after all done
            # use the attributes to calculate the dice score for each threshold
            predictions = torch.cat(self.predictions, dim=0)
            targets = torch.cat(self.targets, dim=0)

            # first apply sigmoid to apply correct threshold
            predictions = torch.sigmoid(predictions)

            # try all thresholds
            for threshold in thresholds:
                threshold = torch.tensor(threshold)

                # make predictions binary based on threshold            
                pred_th = torch.where(predictions > threshold, 1, 0)

                dice_score = self.dice(pred_th, targets, apply_sigmoid=False).cpu()
                print('TH: ', threshold, ' - Dice: ', dice_score)
                scores.append(dice_score)

            # get the index of the best dice score
            index_best = torch.argmax(torch.tensor(scores))

            best_dice = torch.round(scores[index_best].clone(), decimals=3)
            best_threshold = thresholds[index_best]

            print(f'Best Threshold: {best_threshold} - Best Dice Score: {best_dice}')

            # Log the best dice valid        
            self.log_dict({'Best_Threshold': torch.tensor(best_threshold), 'Best_Dice': best_dice},
                           on_step=False, on_epoch=True, prog_bar=True, logger=True)

            # reset after each epoch
            self.predictions = []
            self.targets = []
        
    def configure_optimizers(self):
        optim = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        # needs params: total steps or steps per epoch and epochs
        total_steps = len(self.trainer.datamodule.train_dataloader()) * self.epochs
        scheduler = OneCycleLR(optimizer=optim,
                               max_lr=self.learning_rate,
                               epochs=self.epochs,
                               #steps_per_epoch=10,
                               total_steps = total_steps, #epochs*10,
                               pct_start=self.pct_increase,)
                               #verbose=True)
        return [optim], [{"scheduler": scheduler, "interval": "step"}]

# Train / validate 3 models with the random weights

In [24]:
%%time

for i in range(0, 3):
    MODEL = NN(model=model, learning_rate=learning_rate, calculate_threshold=True)
    MODEL.model.load_state_dict(torch.load(random_weights_path))
    train_ds = Data(paths=train_data[i], transform=train_aug)
    valid_ds = Data(paths=valid_data[i], transform=valid_aug)
    dm = DataModule(train_ds=train_ds, valid_ds=valid_ds)
    logger = TensorBoardLogger(folder_logs, name=name, version=f'Fold_{i+1}')
    trainer = pl.Trainer(accelerator='gpu', devices=[0], max_epochs=epochs, precision='16-mixed', logger=logger)
    trainer.fit(MODEL, dm)
    path_save_weights = os.path.join(folder, 'Weights', f'Model_{i + 1}_{name}.pth')
    torch.save(MODEL.model.state_dict(), path_save_weights)
    gc.collect()
    torch.cuda.empty_cache()

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | Unet              | 34.6 M
1 | dice      | CustomDice        | 0     
2 | BCE       | BCEWithLogitsLoss | 0     
3 | dice_loss | DiceLoss          | 0     
------------------------------------------------
34.6 M    Trainable params
0         Non-trainable params
34.6 M    Total params
138.209   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/olli/anaconda3/envs/light/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


TH:  tensor(0.2000, dtype=torch.float64)  - Dice:  tensor(0.1853)
TH:  tensor(0.2500, dtype=torch.float64)  - Dice:  tensor(0.1853)
TH:  tensor(0.3000, dtype=torch.float64)  - Dice:  tensor(0.1853)
TH:  tensor(0.3500, dtype=torch.float64)  - Dice:  tensor(0.1853)
TH:  tensor(0.4000, dtype=torch.float64)  - Dice:  tensor(0.1853)
TH:  tensor(0.4500, dtype=torch.float64)  - Dice:  tensor(0.1853)
TH:  tensor(0.5000, dtype=torch.float64)  - Dice:  tensor(0.1805)
TH:  tensor(0.5500, dtype=torch.float64)  - Dice:  tensor(0.0004)
TH:  tensor(0.6000, dtype=torch.float64)  - Dice:  tensor(0.)
TH:  tensor(0.6500, dtype=torch.float64)  - Dice:  tensor(0.)
TH:  tensor(0.7000, dtype=torch.float64)  - Dice:  tensor(0.)
TH:  tensor(0.7500, dtype=torch.float64)  - Dice:  tensor(0.)
TH:  tensor(0.8000, dtype=torch.float64)  - Dice:  tensor(0.)
Best Threshold: 0.2 - Best Dice Score: 0.1850000023841858


/home/olli/anaconda3/envs/light/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

TH:  tensor(0.2000, dtype=torch.float64)  - Dice:  tensor(0.2893)
TH:  tensor(0.2500, dtype=torch.float64)  - Dice:  tensor(0.3115)
TH:  tensor(0.3000, dtype=torch.float64)  - Dice:  tensor(0.3293)
TH:  tensor(0.3500, dtype=torch.float64)  - Dice:  tensor(0.3449)
TH:  tensor(0.4000, dtype=torch.float64)  - Dice:  tensor(0.3595)
TH:  tensor(0.4500, dtype=torch.float64)  - Dice:  tensor(0.3731)
TH:  tensor(0.5000, dtype=torch.float64)  - Dice:  tensor(0.3877)
TH:  tensor(0.5500, dtype=torch.float64)  - Dice:  tensor(0.4038)
TH:  tensor(0.6000, dtype=torch.float64)  - Dice:  tensor(0.4178)
TH:  tensor(0.6500, dtype=torch.float64)  - Dice:  tensor(0.4282)
TH:  tensor(0.7000, dtype=torch.float64)  - Dice:  tensor(0.4392)
TH:  tensor(0.7500, dtype=torch.float64)  - Dice:  tensor(0.4521)
TH:  tensor(0.8000, dtype=torch.float64)  - Dice:  tensor(0.4617)
Best Threshold: 0.7999999999999998 - Best Dice Score: 0.4620000123977661


Validation: 0it [00:00, ?it/s]

TH:  tensor(0.2000, dtype=torch.float64)  - Dice:  tensor(0.2929)
TH:  tensor(0.2500, dtype=torch.float64)  - Dice:  tensor(0.3155)
TH:  tensor(0.3000, dtype=torch.float64)  - Dice:  tensor(0.3325)
TH:  tensor(0.3500, dtype=torch.float64)  - Dice:  tensor(0.3467)
TH:  tensor(0.4000, dtype=torch.float64)  - Dice:  tensor(0.3594)
TH:  tensor(0.4500, dtype=torch.float64)  - Dice:  tensor(0.3711)
TH:  tensor(0.5000, dtype=torch.float64)  - Dice:  tensor(0.3837)
TH:  tensor(0.5500, dtype=torch.float64)  - Dice:  tensor(0.3978)
TH:  tensor(0.6000, dtype=torch.float64)  - Dice:  tensor(0.4096)
TH:  tensor(0.6500, dtype=torch.float64)  - Dice:  tensor(0.4192)
TH:  tensor(0.7000, dtype=torch.float64)  - Dice:  tensor(0.4299)
TH:  tensor(0.7500, dtype=torch.float64)  - Dice:  tensor(0.4422)
TH:  tensor(0.8000, dtype=torch.float64)  - Dice:  tensor(0.4521)
Best Threshold: 0.7999999999999998 - Best Dice Score: 0.4519999921321869


`Trainer.fit` stopped: `max_epochs=14` reached.


CPU times: user 9h 53min 42s, sys: 53min 12s, total: 10h 46min 54s
Wall time: 56min 34s
